We are going to take a look at 2011 data since certain datasets are not yet available for the 2021 census. It is worth noting that the LSOAs do change between census, they may have different names or change shape or new ones ma be created owing to housing developments. The Office for National Statistics (ONS) [open geography portal](https://geoportal.statistics.gov.uk/) is very useful for searching available datasets. For example they do have data tables to map from 2011 LSOAs to 2021 LSOAs.

We obtained 2011 Lower Super Output Areas (LSOA) boundaries from the ONS [here](https://geoportal.statistics.gov.uk/datasets/ons::lsoa-dec-2011-boundaries-generalised-clipped-bgc-ew-v3-2). These are clipped (don't extend into the sea etc) and generalised to 20m (slightly less detail than the actual boundaries). I downloaded these as geojson since this has a lot of tools that can be worked with, and saved them as a file called `lsoa_boundaries_2011.geojson`.

This is a `62MB` file and we are only really interested in the coastal towns. And we can start with Dover which we can extract and save like this:

```python
import geopandas as gpd

# load the lsoa boundaries
gdf = gpd.read_file('lsoa_boundaries_2011.geojson')

# get the county
gdf['County'] = gdf['LSOA11NM'].str.split(' ', expand=True)[0]

# filter out dover
dover = gdf[gdf['County']=='Dover']

# save
dover.to_file("lsoa_boundaries_dover_2011.geojson", driver='GeoJSON')
```

This then saves the Dover boundaries in `lsoa_boundaries_dover_2011.geojson`. We can then take a look at it using the interactive mapping library `folium`.

In [1]:
import folium
import geopandas as gpd

# load the data
dover = gpd.read_file("lsoa_boundaries_dover_2011.geojson")

# centre of the map is going to be the centroid of our dover LSOAs
# (as lat,lon)
dover = dover.to_crs(epsg=4326)
centroid = dover.unary_union.centroid

# Create a map centered at the given coordinates
m = folium.Map(location=[centroid.y,centroid.x], zoom_start=11)

# Add the data to the map
folium.GeoJson(dover).add_to(m)

# Display the map
display(m)

That's more interesting, we could now take the 2011 [rural-urban classification data for LSOAs](https://geoportal.statistics.gov.uk/datasets/ons::rural-urban-classification-2011-of-lower-layer-super-output-areas-in-england-and-wales-1) and overlay that. For more information about this classification [see this leaflet](https://www.ons.gov.uk/file?uri=/methodology/geography/geographicalproducts/ruralurbanclassifications/2011ruralurbanclassification/rucoaleafletmay2015tcm77406351.pdf). Assume we have downloaded the data and extracted dover again.

In [2]:
import pandas as pd
# load the urban classification
urban_data = pd.read_csv("lsoa_rural_urban_classifications_2011.csv")
# do a left join on our dover data, with the 2011 urban classifications
dover = dover.merge(urban_data,on=['LSOA11CD','LSOA11NM'],how="left")

dover.head()


,FID_x,LSOA11CD,LSOA11NM,LSOA11NMW,BNG_E,BNG_N,LONG,LAT,GlobalID,County,geometry,FID_y,RUC11CD,RUC11
0,23549,E01024190,Dover 006A,Dover 006A,626213,151775,1.238092,51.22033,db125656-6bb0-4bb5-bb4d-d2dc248f6e64,Dover,"POLYGON ((1.21676 51.23868, 1.21896 51.23700, ...",23550,D1,Rural town and fringe
1,23550,E01024191,Dover 006B,Dover 006B,623724,152754,1.203119,51.23010,952f381c-3f0d-4588-932e-3ed7857e2f53,Dover,"POLYGON ((1.20294 51.23334, 1.20565 51.23308, ...",23551,D1,Rural town and fringe
2,23551,E01024192,Dover 006C,Dover 006C,623315,151989,1.196794,51.22339,09beb277-25f6-49a1-a188-6cb09f40a0d9,Dover,"POLYGON ((1.19371 51.22614, 1.19460 51.22521, ...",23211,D1,Rural town and fringe
3,23552,E01024193,Dover 011A,Dover 011A,630808,143122,1.298227,51.14082,8255c69c-394d-44cc-aa30-5dcbfeef7674,Dover,"POLYGON ((1.30054 51.14350, 1.30060 51.14205, ...",23216,C1,Urban city and town
4,23553,E01024194,Dover 011B,Dover 011B,631049,142725,1.301411,51.13715,961fd2bf-8239-4125-95b2-188924547dc5,Dover,"POLYGON ((1.30357 51.13953, 1.30235 51.13843, ...",23221,C1,Urban city and town


Now we want to colour the polygons according to urban classification to get some idea what it looks like:

In [4]:
import json
from ipyleaflet import Map, GeoJSON, GeoData, basemaps, LayersControl, LegendControl, Popup, embed_minimal_html
from ipywidgets import HTML

# Convert the GeoDataFrame to a GeoJSON object
geo_json_data = json.loads(dover.to_json())

# get a easting,northing coordinates 
dover_projected = dover.to_crs("EPSG:27700")


# Now calculate the centroid
dover_centroid = dover_projected.geometry.centroid

dover_centroid = dover_centroid.to_crs(dover.crs)
centroid_lat = dover_centroid.y.mean()
centroid_lon = dover_centroid.x.mean()

# Create the map
m2 = Map(center=(centroid_lat, centroid_lon), zoom=11, basemap=basemaps.OpenStreetMap.Mapnik)

# map the class to a colour
urban_color_map = {
   'Rural village and dispersed' : '#77dd77',
   'Rural town and fringe' : '#fdfd96',
   'Urban city and town' : '#ffb347',
}

from IPython.core.debugger import set_trace

dover['urban_color_map'] = dover.RUC11.map(urban_color_map)

def style_function(feature):
    urban_class_color = dover.loc[dover['LSOA11CD'] == feature['id'], 'urban_color_map'].values[0]
    return {
        'color': 'black', 
        'fillColor': urban_class_color, 
        'opacity':0.5, 
        'weight':1.9, 
        'dashArray':'2', 
        'fillOpacity':0.6
    }

tooltip = None

def hover_handler(feature, **kwargs):
    global tooltip
    
  
    if tooltip:
        m2.remove_layer(tooltip)
        tooltip = None

    properties = kwargs.get('properties', {})
    lon = properties.get('LONG')
    lat = properties.get('LAT')

    msg = HTML()

    msg.value = f"LSOA11CD: {feature['id']}"
    tooltip = Popup(
        location=(lat,lon),
        child=msg,
        close_button=False,
        auto_close=False,
        close_on_escape_key=False
    )
    m2.add_layer(tooltip)
   


geo_data_classified = GeoJSON(
    data=json.loads(dover.set_index('LSOA11CD').to_json()),
    style_callback=style_function,
    hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
    name='LUC11 Classification'
)
#geo_data_classified.on_hover(hover_handler)

m2.add_layer(geo_data_classified)

# Add layer control to the map
m2.add_control(LayersControl())

# Define the legend
legend = LegendControl(urban_color_map, name="Legend", position="bottomright")

# Add the legend to the map
m2.add_control(legend)

# save a copy of the map
embed_minimal_html('map.html', views=[m2])

# Display the map
display(m2)

Map(center=[51.18822949964158, 1.3180700704816783], controls=(ZoomControl(options=['position', 'zoom_in_text',…

When analyzing the urban classifications of Dover, located in Thanet, we examined specific LSOA regions. However, the question arose as to whether these LSOAs truly represent the town of Dover. This question brings up an important point: are the boundaries and units used in our data the most relevant or appropriate for our research question?

The answer is not always clear-cut. LSOAs are administrative areas defined for the purpose of data collection and analysis, but they may not always coincide with the social, cultural, or economic boundaries that define a 'town'. For example, people's perception of the area they consider to be their town may differ from the official LSOA boundaries.

This analysis could be the start of a more profound exploration of what truly defines a town. This could include investigating alternative data sources that may give a different perspective on what constitutes Dover or any other town. It might mean taking into account other types of data such as demographic, socioeconomic or transport data.

Perhaps the town boundaries might need to be defined based on commuting patterns, shared services, or local perceptions. There might be an opportunity here to use more innovative techniques such as machine learning to identify natural clusters within the data, which could serve as alternative definitions of town boundaries.

This exploration serves as a starting point, a launching pad for deeper investigations